# Comp 472 Mini-Project 2

## 1. Game Setup

### Imports

In [57]:
import pprint
from queue import PriorityQueue
import copy
import time
import os
import openpyxl
import pandas as pd

### Car and Board Class

In [58]:
# Definition of car object 
class Car:
    def __init__(self, name, fuel, coordinates, orientation):
        
        self.name = name
        self.fuel = fuel
        self.coordinates = coordinates # list of coordinates that represents its position in the board
        self.orientation = orientation
        
    # Function used to print the information of the car
    def printCarInfo(self):
        print("Name: ", self.name, ", Fuel: ",self.fuel ,", Coordinates: ", self.coordinates, ", Orientation: ", self.orientation)

In [59]:
# Possible letters that could represent cars in the grid
carLetters =   ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

# Definition of board object
class Board:
    
    # initializer
    
    def __init__(self, puzzleLine): # default cost = 0, for successors do parentboard.cost +1
        self.dimension = 6 # dimension
        
        self.parent = None 
        self.grid = [] # 
        self.cars = {} # dictionary of car objects present in the board  {carName: car object}
        self.puzzleLine = puzzleLine # string 

        # f = g + h
        self.cost = 0
        self.heuristic = 0
        self.f = 0
        
        self.successorMove = ""   # "M down 1    99"
        
        # set initial info  ---------------------
        self.fillGrid()
        coordinatesDict = self.setCoordinate()
        fuelDict = self.setInitialFuel()
        self.createCarObjects(coordinatesDict,fuelDict)
                

    # Getters and setters for g, h, f --------------------        
    def setCost(self, cost):
        self.cost = cost
        
    def setHeuristic(self,heuristic):
        self.heuristic = heuristic
    
    def setF(self,cost,heuristic):
        self.f = (cost + heuristic)
        
    def getCost(self):
        return self.cost
    
    def getHeuristic(self):
        return self.heuristic
    
    def getF(self):
        return self.f
            
                
    # Functions to set the initial info ---------------------        
    def fillGrid(self):
        # Creating empty grid
        self.grid = [[0 for x in range(self.dimension)] for x in range(self.dimension)] 

        # Filling grid with puzzle line information (36 characters)
        a = 0    
        for i in range(self.dimension):
            for j in range(self.dimension):
                self.grid[i][j] = self.puzzleLine[a] # walk through up until 6*6 =36 characters
                a += 1

    def setCoordinate(self):
         # Getting coordinates of the cars in the grid
        coordinatesDict = {}
        for letter in carLetters:
            if letter in self.puzzleLine: # If car letter exists in the board
                coordinatesDict[letter] = [[x, y] for x, li in enumerate(self.grid) for y, val in enumerate(li) if val==letter]
        return coordinatesDict
                    
    def setInitialFuel(self):
         # Checking for initial fuel units for the cars in the board \for the specific puzzle (Default fuel units = 100)
        fuelDict = {}
        fuelInfoFromPuzzleLine = self.puzzleLine[(self.dimension*self.dimension):].strip()
        if fuelInfoFromPuzzleLine != "": # If additional info exists after the puzzle line, then initial fuel info exists
            initialFuelInfo = fuelInfoFromPuzzleLine.split()
            for fuel in initialFuelInfo:
                fuelDict[fuel[0]] = int(fuel[1:])
        return fuelDict

    
    def createCarObjects(self,coordinatesDict,fuelDict):
                # Creating car object with obtained information
        for carName in coordinatesDict:

            # Checking the orientation of each car in the board
            # If the x of the coordinates are equal, then horizontal. If not, then vertical
            if coordinatesDict[carName][0][0] == coordinatesDict[carName][1][0]:
                orientation = "horizontal"
            else:
                orientation = "vertical"

            if carName in fuelDict:
                self.cars[carName] = Car(carName, fuelDict[carName], coordinatesDict[carName], orientation)
            else:
                self.cars[carName] = Car(carName, 100, coordinatesDict[carName], orientation)
     
    
    # Functions to print information -------------------------
    # Function to print the initial Grid from string       
    def returnPuzzleLine(self):
        grid = ""
        a = 0
        for i in range(self.dimension):
            grid = grid + self.puzzleLine[a:(a+6)]+"\n"
            a += 6
        grid = "! "+ self.puzzleLine[37:] + "\n" + grid
        return grid
              
    # Function to print the current board
    def printGrid(self):
        pprint.pprint(self.grid)
        
    # Function to print board info
    def printBoardInfo(self):
        print(self.puzzleLine)
        print("Board: ")
        self.printGrid()
        for each in (self.cars).values():
            each.printCarInfo()
           
    # Function to retrieve the puzzle line of the current board
    def getPuzzleLine(self):
        successorStr = ""
        for line in self.grid:
            for char in line:
                successorStr += char

        for car in self.cars.values():
            if car.fuel < 100:
                successorStr += (" " + car.name + str(car.fuel)) 
        return successorStr


    # Function to generate the possible states that the board can be in depending on the possible moves of one car    
    def generateSuccessorBoards(self):
        successors = [] # List of successor board objects

        for car in self.cars.values():
            # horizontal -- ======================
            if car.orientation == "horizontal":

                positionsMoved = 0
                # Check left direction ===========
                left = car.coordinates[0][:] # copy most left coordinate of this car
                while(left[1] > 0): # until we reach left edge of board
                    left[1] -= 1 # check one cell to the left
                    
                    if self.grid[left[0]][left[1]] == '.': # if that cell is empty
                        positionsMoved += 1 
                        # If there is not enough fuel to move, then skip and do not generate new successors
                        if (car.fuel) - positionsMoved < 0:
                            continue
                        else:
                            newCoordinates = [row[:] for row in car.coordinates]
                            # [[x,y],[z,t]]
                            for each in newCoordinates:
                                each[1] -= positionsMoved
                            
                            # create one child
                            child = self.createOneSuccessor(newCoordinates, positionsMoved, car.name, car.coordinates,"left" )
                               
                            # append to successors list <- contains child objects
                            successors.append(child)
                    else:
                        break

                # Check right direction ===========
                positionsMoved = 0
                right = car.coordinates[-1][:] # copy most right coordinate of this car

                while(right[1] < 5): # until we reach right edge of board
                    right[1] += 1 # check one cell to right

                    if self.grid[right[0]][right[1]] == '.':
                        
                        positionsMoved += 1 
                        # If there is not enough fuel to move, then skip and do not generate new successors
                        if (car.fuel) - positionsMoved < 0:
                            continue
                        else:
                            newCoordinates = [row[:] for row in car.coordinates]
                            # [[x,y],[z,t]]
                            for each in newCoordinates:
                                each[1] += positionsMoved
                            
                            # create one child
                            child = self.createOneSuccessor(newCoordinates, positionsMoved, car.name, car.coordinates ,"right")
                                
                            # append to successors list <- contains child objects
                            successors.append(child)
                    else:
                        break

            # vertical | ======================
            if car.orientation == "vertical":
                
                # Check up direction ===========
                positionsMoved = 0
                up = car.coordinates[0][:]
                
                while(up[0] > 0):
                    up[0] -= 1
                    if self.grid[up[0]][up[1]] == '.':
                        positionsMoved += 1 
                        # If there is not enough fuel to move, then skip and do not generate new successors
                        if (car.fuel) - positionsMoved < 0:
                            continue
                        else:
                            newCoordinates = [row[:] for row in car.coordinates]
                            # [[x,y],[z,t]]
                            for each in newCoordinates:
                                each[0] -= positionsMoved
                            
                            # create one child
                            child = self.createOneSuccessor(newCoordinates, positionsMoved, car.name, car.coordinates,"up" )
                             
                            # append to successors list <- contains child objects
                            successors.append(child)
                    else:
                        break

                # Check down direction ===========
                positionsMoved = 0
                down = car.coordinates[-1][:]
                
                while(down[0] < 5):
                    down[0] += 1
                    if self.grid[down[0]][down[1]] == '.':
                        positionsMoved += 1 
                        # If there is not enough fuel to move, then skip and do not generate new successors
                        if (car.fuel) - positionsMoved < 0:
                            continue
                        else:
                            newCoordinates = [row[:] for row in car.coordinates]
                            # [[x,y],[z,t]]
                            for each in newCoordinates:
                                each[0] += positionsMoved
                            
                            # create one child
                            child = self.createOneSuccessor(newCoordinates, positionsMoved, car.name, car.coordinates, "down" )
                            
                            # append to successors list <- contains child objects
                            successors.append(child)
                    else:
                        break

        return successors 
    
    
    # helper function (used inside updateBoardAndCarInfo function)
    def updateGrid(self, carName, oldCoordinates, newCoordinates ):
        
        for each in oldCoordinates:
            self.grid[each[0]][each[1]] = '.'
        for each in newCoordinates:
            self.grid[each[0]][each[1]] = carName


    # Function to one single successor wiht its information
    def createOneSuccessor(self, newCoordinates, positionsMoved, carName, oldCoordinates, direction ):
        child = copy.deepcopy(self) # deepcopy of current board
        child.parent = self
        child.cars[carName].coordinates = newCoordinates # update car coordinate
        child.updateGrid(carName, oldCoordinates, newCoordinates ) # update gird
        child.cars[carName].fuel -= positionsMoved # update fuel
        child.cost = self.cost + 1 
        child.puzzleLine = child.getPuzzleLine()
        child.successorMove = "{}{:>6}{:>2}{:>7}".format(carName, direction, str(positionsMoved), 
                                                         str(child.cars[carName].fuel))
        child.f = 0
        child.heuristic = 0
        return child
    
    
    # Function that checks if the car is A    
    def isGoal(self):
        
        ambulance = self.cars['A']
        if ambulance.coordinates[-1] == [2,5]:
            return True
        else:
            return False
    
    # Function that checks if the car is at the exit
    def leaveGridIfAtExit(self):
        
        for car in (self.cars).values():
            if car.orientation == "horizontal":
                if car.coordinates[-1] == [2,5]:
                    # exit
                    for each in car.coordinates:
                        self.grid[each[0]][each[1]] = '.' 
                    # Removes the exited car from the list of cars      
                    (self.cars).pop(car.name)
                break
            else:
                pass

## 2.1 State Space Search

In [60]:
# helper function used in all three search
def getSolutionPath(sol, CLOSED, Root):
    solutionPath= []
    solutionPathStr = ""
    isSolFound = sol
    
    # If sol is 0, then there is no solution path, return solutionPath = []
    if sol == 0:
        return solutionPath
        
    goalState =  CLOSED[-1] # last element is our goal state
    
    # We backtrack starting with the goalState and using its parents until we reach the Root
    while True:
        if goalState == Root:
            solutionPath.reverse()
            solutionPath.append(solutionPathStr)
            return solutionPath
        else:
            solutionPathStr = goalState.successorMove[:9]+"; " + solutionPathStr
            solutionPath.append(goalState.successorMove+" "+goalState.puzzleLine)
            goalState = goalState.parent

### 2.1.1 Uniform Cost Search (UCS)

In [61]:
def UniformCostSearch(Root):
    CLOSED = []
    OPEN = []
    
    # Append the rootState (initial state) to the OPEN list
    OPEN.append(Root)
    
    # Check if the rootState is goal
    if Root.isGoal():
        OPEN.pop(0)
        CLOSED.append(Root)
        return CLOSED, getSolutionPath(1,CLOSED, Root)
    
    # Begin search
    while True:
        # If the OPEN list is empty, then there is no solution found
        if not OPEN:
            return CLOSED, getSolutionPath(0,CLOSED, Root)
        
        # "Visit" the first board in the OPEN list and add it to the CLOSED list
        parentBoard = OPEN.pop(0)
        CLOSED.append(parentBoard)

        # Checking if the current board we are visiting is the goal state
        if parentBoard.isGoal():
            return CLOSED, getSolutionPath(1,CLOSED, Root)
        else: 
            parentBoard.leaveGridIfAtExit()
            
        # Generate possible successor moves for the cars on the board
        successors = parentBoard.generateSuccessorBoards()
        
        for child in successors:
            
            # Checking if the current successor board is already present in the CLOSED list
            if(not any(child.grid == visitedBoard.grid for visitedBoard in CLOSED)):
                # If the OPEN list is empty, we will append the first child
                if not OPEN:
                    OPEN.append(child)
                else:    
                    # Checking if the current successor board is already present in the OPEN list
                    # If it is and the cost of the one in the OPEN list is greater, 
                    # then we retain the value to replace it in the OPEN list
                    toBeReplaced = [x for x in OPEN if (child.grid == x.grid and child.f < x.f)]
                    
                    # If toBeReplaced has value, it means we found a value to replace
                    if toBeReplaced:
                        OPEN.remove(toBeReplaced[0])
                        OPEN.append(child)
                        
                     # If it is a new successor, we append it to the list   
                    elif any((child.grid == toBeVisited.grid) for toBeVisited in OPEN):
                        continue
                    else:
                        OPEN.append(child)
        
        # Sort this list by cost (priority by cost)
        OPEN.sort(key=lambda x: x.cost, reverse=False)


### Heuristics Experimented for GBFS and Algorithm A

#### h1: The number of blocking vehicles

In [62]:
def heuristicOne(board):
    carsOnRight = []
    
    ambulance = board.cars['A']
    rightOfA = ambulance.coordinates[-1]  # right most coordiante of Ambulance [x,y]
    i = rightOfA[1]
        
    while(i < 5): # until we reach right edge of Grid # y coordinate
        i += 1 # check one cell to right
        cellContent = board.grid[rightOfA[0]][i]  # 'M' or '.' ..
        
        if cellContent == '.':
            continue
        else:
            if cellContent not in carsOnRight:
                carsOnRight.append(cellContent) # append the name
    
    return len(carsOnRight)

#### h2: The number of blocking positions

In [63]:
def heuristicTwo(board):
    carsOnRight = []
    
    ambulance = board.cars['A']
    rightOfA = ambulance.coordinates[-1]  # right most coordiante of Ambulance [x,y]
    i = rightOfA[1]
        
    while(i < 5): # until we reach right edge of Grid # y coordinate
        i += 1 # check one cell to right
        cellContent = board.grid[rightOfA[0]][i]  # 'M' or '.' ..
        
        if cellContent == '.':
            continue
        else:
            carsOnRight.append(cellContent) # append the name
    
    return len(carsOnRight)

#### h3: The value of h1 multiplied by a constant λ of your choice, where λ > 1 (Chosen λ = 5)

In [64]:
def heuristicThree(board):
    alpha = 5
    h1 = heuristicOne(board)
    return h1*alpha

#### h4: h1 + 1 (each blocking vehicle has to move at least once, and then the ambulance still needs a cost of 1 to exit)

In [65]:
def heuristicFour(board):
    h1 = heuristicOne(board)
    return h1+1

### 2.1.2 Greedy Best First Search (GBFS)

In [66]:
def GreedyBestFirstSearch(Root, heuristic):
    CLOSED = []
    OPEN = []
    
    # Heuristic chosen to be applied on the initial board
    if heuristic == 1:
        h1 = heuristicOne(Root)
        Root.setHeuristic(h1)
    elif heuristic == 2:
        h2 = heuristicTwo(Root)
        Root.setHeuristic(h2)
    elif heuristic == 3:
        h3 = heuristicThree(Root)
        Root.setHeuristic(h3)
    elif heuristic == 4:
        h4 = heuristicFour(Root)
        Root.setHeuristic(h4)
    else:
        # If user inputs an invalid heuristic number
        return "Heuristic does not exist. Try again with valid heuristic."
        
        
    # Append the rootState (initial state) to the OPEN list
    OPEN.append(Root)
    
    # Check if the rootState is goal
    if Root.isGoal():
        OPEN.pop(0)
        CLOSED.append(Root)
        return CLOSED, getSolutionPath(1, CLOSED, Root)
    
    # Begin search
    while True:
        
        # If the OPEN list is empty, then there is no solution found
        if not OPEN:
            return CLOSED, getSolutionPath(0, CLOSED, Root)
        
        # "Visit" the first board in the OPEN list and add it to the CLOSED list
        parentBoard = OPEN.pop(0)
        CLOSED.append(parentBoard)

        # Checking if the current board we are visiting is the goal state
        if parentBoard.isGoal():
            return CLOSED, getSolutionPath(1, CLOSED, Root)
        else: 
            parentBoard.leaveGridIfAtExit() # remove cars from Board if at exit
            
        # Generate possible successor moves for the cars on the board
        successors = parentBoard.generateSuccessorBoards()
        
        for child in successors:
            
            # Heuristic chosen to be applied on the initial board
            if heuristic == 1:
                h1 = heuristicOne(child)
                child.setHeuristic(h1)
            elif heuristic == 2:
                h2 = heuristicTwo(child)
                child.setHeuristic(h2)
            elif heuristic == 3:
                h3 = heuristicThree(child)
                child.setHeuristic(h3)
            elif heuristic == 4:
                h4 = heuristicFour(child)
                child.setHeuristic(h4)        
        
            # Checking if the current successor board is already present in the CLOSED list
            if(not any(child.grid == visitedBoard.grid for visitedBoard in CLOSED)):
                # If the OPEN list is empty, we will append the first child
                if not OPEN:
                    OPEN.append(child)
                else:    
                    # Checking if the current successor board is already present in the OPEN list
                    # If it is and the cost of the one in the OPEN list is greater, 
                    # then we retain the value to replace it in the OPEN list
                    toBeReplaced = [x for x in OPEN if (child.grid == x.grid and child.f < x.f)]
                    
                    # If toBeReplaced has value, it means we found a value to replace
                    if toBeReplaced:
                        OPEN.remove(toBeReplaced[0])
                        OPEN.append(child)
                        
                     # If it is a new successor, we append it to the list   
                    elif any((child.grid == toBeVisited.grid) for toBeVisited in OPEN):
                        continue
                    else:
                        OPEN.append(child)

        # Sort this list by the heuristic applied (priority by heuristic applied)
        OPEN.sort(key=lambda x: x.heuristic, reverse=False)


### 2.1.3 Algorithm A/A*

In [67]:
def AlgorithmA(Root, heuristic):
    CLOSED = []
    OPEN = []
    count = 0
    
    # Heuristic chosen to be applied on the initial board
    if heuristic == 1:
        h1 = heuristicOne(Root)
        Root.setHeuristic(h1)
        Root.setF(Root.getCost(), h1)
    elif heuristic == 2:
        h2 = heuristicTwo(Root)
        Root.setHeuristic(h2)
        Root.setF(Root.getCost(), h2)
    elif heuristic == 3:
        h3 = heuristicThree(Root)
        Root.setHeuristic(h3)
        Root.setF(Root.getCost(), h3)
    elif heuristic == 4:
        h4 = heuristicFour(Root)
        Root.setHeuristic(h4)
        Root.setF(Root.getCost(), h4)
    else:
        # If user inputs an invalid heuristic number
        return "Heuristic does not exist. Try again with valid heuristic."
        
        
    # Append the rootState (initial state) to the OPEN list
    OPEN.append(Root)
    
    # Check if the rootState is goal
    if Root.isGoal():
        OPEN.pop(0)
        CLOSED.append(Root)
        return CLOSED, getSolutionPath(1, CLOSED, Root)
    
    # Begin search
    while True:
        
        # If the OPEN list is empty, then there is no solution found
        if not OPEN:
            return CLOSED, getSolutionPath(0, CLOSED, Root)
        
        # "Visit" the first board in the OPEN list and add it to the CLOSED list
        parentBoard = OPEN.pop(0)
        CLOSED.append(parentBoard)

        # Checking if the current board we are visiting is the goal state
        if parentBoard.isGoal():
            return CLOSED, getSolutionPath(1, CLOSED, Root)
        else: 
            parentBoard.leaveGridIfAtExit() # remove cars from Board if at exit
            
        # Generate possible successor moves for the cars on the board
        successors = parentBoard.generateSuccessorBoards()
        
        for child in successors:
            
            # Heuristic chosen to be applied on the initial board
            if heuristic == 1:
                h1 = heuristicOne(child)
                child.setHeuristic(h1)
                child.setF(child.getCost(), h1)
            elif heuristic == 2:
                h2 = heuristicTwo(child)
                child.setHeuristic(h2)
                child.setF(child.getCost(), h2)
            elif heuristic == 3:
                h3 = heuristicThree(child)
                child.setHeuristic(h3)
                child.setF(child.getCost(), h3)
            elif heuristic == 4:
                h4 = heuristicFour(child)
                child.setHeuristic(h4)
                child.setF(child.getCost(), h4)     
                
            # Checking if the current successor board is already present in the CLOSED list
            if(not any(child.grid == visitedBoard.grid for visitedBoard in CLOSED)):
                # If the OPEN list is empty, we will append the first child
                if not OPEN:
                    OPEN.append(child)
                else:    
                    # Checking if the current successor board is already present in the OPEN list
                    # If it is and the cost of the one in the OPEN list is greater, 
                    # then we retain the value to replace it in the OPEN list
                    toBeReplaced = [x for x in OPEN if (child.grid == x.grid and child.f < x.f)]
                    
                    # If toBeReplaced has value, it means we found a value to replace
                    if toBeReplaced:
                        OPEN.remove(toBeReplaced[0])
                        OPEN.append(child)
                        
                     # If it is a new successor, we append it to the list   
                    elif any((child.grid == toBeVisited.grid) for toBeVisited in OPEN):
                        continue
                    else:
                        OPEN.append(child)
        
        # Sort this list by f (g+h) (priority by f (g+h))
        OPEN.sort(key=lambda x: x.f, reverse=False)


## 2.2 Read Input File

In [68]:
def readPuzzles(file_path):
    hashtag = "#"
    with open(file_path) as file:
        puzzles = [line.rstrip() for line in file]
        puzzles = list(filter(None, puzzles))

        for line in puzzles.copy():
            if hashtag in line:
                puzzles.remove(line)
    return puzzles

## 2.3 Save output files

### 2.3.1 Solution Files

In [69]:
def saveSolutionFile(fileName, board,runTime, solutionPath, CLOSED):
     
    try:
        # Make directory if it does not already exist
        path = "solution_files"
        if not os.path.exists(path):
           os.makedirs(path)
        
        with open(path + "/" + fileName + ".txt", "w") as f:
            
            f.writelines(["--------------------------------------------------------------------------------\n\n",
                          "Initial board configuration: " + board.puzzleLine + "\n\n",
                          CLOSED[0].returnPuzzleLine() + "\n"])
            f.write("Car fuel available: ")

            for each in board.cars:
                if board.cars[each] == list(board.cars.values())[-1]:
                    f.write(each + ": " + str(board.cars[each].fuel) + "\n")
                else:
                    f.write(each + ": " + str(board.cars[each].fuel) + ", ")

            if not solutionPath: # solution path is empty. solution not found:
                f.writelines(["\nSorry, could not solve the puzzle as specified.", 
                              "\nError: no solution found", 
                              "\n\nRuntime: ", str(runTime)," seconds", 
                              "\n--------------------------------------------------------------------------------"])
            else:
                f.writelines(["\nRuntime: ", str(runTime)," seconds", 
                              "\nSearch path length: " + str(len(CLOSED)) + " states", 
                              "\nSolution path length: " + str(len(solutionPath[:-1])) + " moves", 
                              "\nSolution path: "+ str(solutionPath[-1]) + "\n\n"])

                for each in solutionPath[:-1]:
                    f.write(each + "\n")

                f.writelines(["\n" + CLOSED[-1].returnPuzzleLine(), 
                              "\n--------------------------------------------------------------------------------"])
                
    except IOError:
        print("Error: IOError")
        return 0
    finally:
        f.close()

### 2.3.2 Search Files

In [70]:
def saveSearchFile(fileName, CLOSED):
    
    try: 
        # Make directory if it does not already exist
        path = "search_files"
        if not os.path.exists(path):
           os.makedirs(path)
        
        with open(path + "/" + fileName + ".txt", "w") as f:
            
            if fileName[0] == "u":
                for board in CLOSED:
                    print("{}{:>3}{:>3}".format(board.getCost(), board.getCost(), 0), end = " ", file = f)
                    print(board.puzzleLine, file = f)
            elif fileName[0] == "g":     
                for board in CLOSED:
                    print("{}{:>3}{:>3}".format(board.getHeuristic(), 0, board.getHeuristic()), end = " ", file = f)
                    print(board.puzzleLine, file = f)
            elif fileName[0] == "a":    
                for board in CLOSED:
                    print("{}{:>3}{:>3}".format(board.getF(), board.getCost(), board.getHeuristic()), end = " ", file = f)
                    print(board.puzzleLine, file = f) 
                
    except IOError:
        print("Error: IOError")
        return 0   

## Game Engine

In [71]:
def gameEngine(file_path):
    
    # EXCEL OUTPUT===================================
    puzzleNum= []
    algo = []
    heuristic = []
    lenSolutionPath= []
    lenClose = []
    runTime = []
    # ================================================
    
    i=1
    puzzles = readPuzzles(file_path)

    for each in puzzles:
        myBoard = Board(each)
        
        # === UCS ===
        start_time = time.time()
        ucs_CLOSED, ucs_SolutionPath = UniformCostSearch(myBoard)
        run_time = round(time.time() - start_time, 2)
        
        # TXT OUTPUT +++++++++++++++++++++++++++++++++++
        ucs_SearchFileName = "ucs-search-" + str(i)
        ucs_SolutionFileName = "ucs-sol-" + str(i)
        saveSearchFile(ucs_SearchFileName, ucs_CLOSED)
        saveSolutionFile(ucs_SolutionFileName, myBoard, run_time, ucs_SolutionPath, ucs_CLOSED)
        # +++++++++++++++++++++++++++++++++++++++++++++++
        
        # EXCEL OUTPUT===================================
        puzzleNum.append(i)
        algo.append("UCS")
        heuristic.append("N/A")
        lenSolutionPath.append(len(ucs_SolutionPath[:-1]))
        lenClose.append(len(ucs_CLOSED))
        runTime.append(run_time)
        # ================================================
        
        
        for j in range(1, 5):
        
            # === GBFS ===
            start_time = time.time()
            gbfs_CLOSED, gbfs_SolutionPath = GreedyBestFirstSearch(myBoard, j)
            run_time = round(time.time() - start_time, 2)
        
            # TXT OUTPUT +++++++++++++++++++++++++++++++++++
            gbfs_SearchFileName = "gbfs-h" + str(j) + "-search-" + str(i)
            gbfs_SolutionFileName = "gbfs-h" + str(j) + "-sol-" + str(i)
            saveSearchFile(gbfs_SearchFileName, gbfs_CLOSED)
            saveSolutionFile(gbfs_SolutionFileName, myBoard, run_time, gbfs_SolutionPath, gbfs_CLOSED)
            # +++++++++++++++++++++++++++++++++++++++++++++++
            
            # EXCEL OUTPUT===================================
            puzzleNum.append(i)
            algo.append("GBFS")
            heuristic.append("h" + str(j))
            lenSolutionPath.append(len(gbfs_SolutionPath[:-1]))
            lenClose.append(len(gbfs_CLOSED))
            runTime.append(run_time)
            # ================================================
            

        for j in range(1, 5):
            
            # === Algo A ===
            start_time = time.time()
            a_CLOSED, a_SolutionPath = AlgorithmA(myBoard, j)
            runtime = round(time.time() - start_time, 2)
        
            # TXT OUTPUT +++++++++++++++++++++++++++++++++++
            a_SearchFileName = "a-h"+str(j)+"-search-"+str(i)
            a_SolutionFileName = "a-h"+str(j)+"-sol-"+str(i)
            saveSearchFile(a_SearchFileName, a_CLOSED)
            saveSolutionFile(a_SolutionFileName, myBoard, run_time, a_SolutionPath, a_CLOSED)
            # +++++++++++++++++++++++++++++++++++++++++++++++
            
            # EXCEL OUTPUT===================================
            puzzleNum.append(i)
            algo.append("A/A*")
            heuristic.append("h" + str(j))
            lenSolutionPath.append(len(a_SolutionPath[:-1]))
            lenClose.append(len(a_CLOSED))
            runTime.append(run_time)
            # ================================================
  
        i +=1
        
    # EXCEL OUTPUT===================================
    df = pd.DataFrame({'Puzzle Number': puzzleNum, 'Algorithm': algo, 'Heuristic': heuristic ,'Length of the Solution': lenSolutionPath, 'Length of the Search Path': lenClose,'Execution Time (in seconds)': runTime }, 
                      columns=['Puzzle Number', 'Algorithm', 'Heuristic', 'Length of the Solution', 'Length of the Search Path', 'Execution Time (in seconds)'])
    writer = pd.ExcelWriter('50-random-puzzles.xlsx')
    df.to_excel(writer, sheet_name='50-puzzles', index=False)
    
    # Auto-adjust columns' width
    for column in df:
        column_width = max(df[column].astype(str).map(len).max(), len(column))
        col_idx = df.columns.get_loc(column)
        writer.sheets['50-puzzles'].set_column(col_idx, col_idx, column_width)
    
    writer.save()
    # ================================================

### Running the game

In [72]:
# Different file paths
sample_input_path = "SampleInputOutput/Sample/sample-input.txt"
fifty_puzzles_path = "50-random-puzzles.txt"
filePath = "ourInput.txt"

gameEngine(filePath)

C:\Users\Lisa Duong\AppData\Local\Temp\ipykernel_13328\3116245505.py:101: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
